In [1]:
from typing import Callable

import torch
import numpy
import matplotlib.pyplot as plt

from utilities import CostParameters, least_square_cost, ridge_cost

In [2]:
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    torch.set_default_device("cuda")

In [3]:
TEST_DATA_SIZE = 100
X = numpy.sort(numpy.random.rand(TEST_DATA_SIZE, 1), axis=0)
Y = numpy.random.rand(TEST_DATA_SIZE, 1)
Z = numpy.random.rand(TEST_DATA_SIZE, 1)

In [4]:
input_layer = 2
output_layer = 1
learning_rate = 1e-5

def train(hidden_layer: int, epochs: int, cost_func: Callable[[CostParameters], torch.Tensor], cost_func_name: str, print_debug_info: bool = False) -> None:
    x_tensor = torch.tensor(numpy.concatenate((X, Y), axis=1))
    y_tensor = torch.tensor(Z)

    weights_layer1 = torch.rand(input_layer, hidden_layer, requires_grad=True)
    biases_layer1 = torch.rand(hidden_layer, requires_grad=True)
    weights_layer2 = torch.rand(hidden_layer, output_layer, requires_grad=True)
    biases_layer2 = torch.rand(output_layer, requires_grad=True)

    def predict() -> torch.Tensor:
        return (x_tensor @ weights_layer1 + biases_layer1) @ weights_layer2 + biases_layer2

    for i in range(epochs):
        predicted = predict()

        cost_params = CostParameters(y_tensor, predicted, [weights_layer1, weights_layer2], [biases_layer1, biases_layer2])
        loss = cost_func(cost_params)

        if print_debug_info:
            iteration_id = i + 1
            if iteration_id % 500 == 0:
                print(f"Info at epoch #{iteration_id}:")
                print(f"Loss: {loss.item()}")

        loss.backward()

        with torch.no_grad():
            weights_layer1 -= learning_rate * weights_layer1.grad
            biases_layer1 -= learning_rate * biases_layer1.grad
            weights_layer2 -= learning_rate * weights_layer2.grad
            biases_layer2 -= learning_rate * biases_layer2.grad

            weights_layer1.grad = None
            biases_layer1.grad = None
            weights_layer2.grad = None
            biases_layer2.grad = None

    final_prediction = numpy.array(predict().detach().cpu())

    sub_plot = plt.figure().add_subplot(projection="3d")
    sub_plot.set_title(f"Training a 3D regression model with {hidden_layer} neurons within the hidden layer after {epochs} epochs")

    sub_plot.plot_trisurf(X.reshape(TEST_DATA_SIZE), Y.reshape(TEST_DATA_SIZE), final_prediction.reshape(TEST_DATA_SIZE), color="g", alpha=0.6)
    sub_plot.scatter(X, Y, Z)

    plt.savefig(f"plots/linear_regression_3d_{hidden_layer}_{epochs}_{cost_func_name}.png", bbox_inches='tight')
    plt.close()

In [5]:
train(1, 5_000, least_square_cost, "lsc", print_debug_info=True)

Info at epoch #500:
Loss: 8.44112511169628
Info at epoch #1000:
Loss: 8.336761062221651
Info at epoch #1500:
Loss: 8.311240155163272
Info at epoch #2000:
Loss: 8.28867621794765
Info at epoch #2500:
Loss: 8.267530985649843
Info at epoch #3000:
Loss: 8.247664488501211
Info at epoch #3500:
Loss: 8.22897432488051
Info at epoch #4000:
Loss: 8.211368502078075
Info at epoch #4500:
Loss: 8.194763852655784
Info at epoch #5000:
Loss: 8.17908503082807


![Linear regression 3D model with 1 hidden neuron and 5000 epochs](./plots/linear_regression_3d_1_5000_lsc.png)

In [6]:
train(10, 5_000, least_square_cost, "lsc", print_debug_info=True)

Info at epoch #500:
Loss: 10.417270640661464
Info at epoch #1000:
Loss: 9.583224863334525
Info at epoch #1500:
Loss: 9.003601615682273
Info at epoch #2000:
Loss: 8.595143032040271
Info at epoch #2500:
Loss: 8.304408147965058
Info at epoch #3000:
Loss: 8.09596212578579
Info at epoch #3500:
Loss: 7.945723544461366
Info at epoch #4000:
Loss: 7.8370194927169825
Info at epoch #4500:
Loss: 7.758144013702089
Info at epoch #5000:
Loss: 7.700792134574388


![Linear regression 3D model with 10 hidden neuron and 5000 epochs](./plots/linear_regression_3d_10_5000_lsc.png)

In [7]:
train(100, 5_000, least_square_cost, "lsc", print_debug_info=True)

Info at epoch #500:
Loss: 9.994999001466187
Info at epoch #1000:
Loss: 7.657365530900432
Info at epoch #1500:
Loss: 7.552061725988223
Info at epoch #2000:
Loss: 7.546373240467762
Info at epoch #2500:
Loss: 7.546053634457682
Info at epoch #3000:
Loss: 7.546035543883368
Info at epoch #3500:
Loss: 7.546034518516761
Info at epoch #4000:
Loss: 7.546034460385048
Info at epoch #4500:
Loss: 7.546034457089201
Info at epoch #5000:
Loss: 7.5460344569023405


![Linear regression 3D model with 100 hidden neuron and 5000 epochs](./plots/linear_regression_3d_100_5000_lsc.png)

In [8]:
train(1, 5_000, ridge_cost(least_square_cost, 0.5), "ridge+lsc", print_debug_info=True)

Info at epoch #500:
Loss: 8.366360440461763
Info at epoch #1000:
Loss: 8.214376448000761
Info at epoch #1500:
Loss: 8.193699162143636
Info at epoch #2000:
Loss: 8.177169266118932
Info at epoch #2500:
Loss: 8.162976687855464
Info at epoch #3000:
Loss: 8.150644795311079
Info at epoch #3500:
Loss: 8.139855253479881
Info at epoch #4000:
Loss: 8.130371607745356
Info at epoch #4500:
Loss: 8.122007632854977
Info at epoch #5000:
Loss: 8.114611124870041


![Linear regression 3D model with 1 hidden neuron and 5000 epochs](./plots/linear_regression_3d_1_5000_ridge+lsc.png)

In [9]:
train(10, 5_000, ridge_cost(least_square_cost, 0.5), "ridge+lsc", print_debug_info=True)

Info at epoch #500:
Loss: 16.618846123947968
Info at epoch #1000:
Loss: 14.155751850433887
Info at epoch #1500:
Loss: 12.55975848988581
Info at epoch #2000:
Loss: 11.505181517426934
Info at epoch #2500:
Loss: 10.79994010852194
Info at epoch #3000:
Loss: 10.324817737828445
Info at epoch #3500:
Loss: 10.003219424600974
Info at epoch #4000:
Loss: 9.784816166164296
Info at epoch #4500:
Loss: 9.636072697131489
Info at epoch #5000:
Loss: 9.534458172102852


![Linear regression 3D model with 10 hidden neuron and 5000 epochs](./plots/linear_regression_3d_10_5000_ridge+lsc.png)

In [10]:
train(100, 5_000, ridge_cost(least_square_cost, 0.5), "ridge+lsc", print_debug_info=True)

Info at epoch #500:
Loss: 15.422140895928
Info at epoch #1000:
Loss: 13.004092408880584
Info at epoch #1500:
Loss: 12.894201506187418
Info at epoch #2000:
Loss: 12.88643261473776
Info at epoch #2500:
Loss: 12.883665420964299
Info at epoch #3000:
Loss: 12.881152008859303
Info at epoch #3500:
Loss: 12.87865163804749
Info at epoch #4000:
Loss: 12.876151941778994
Info at epoch #4500:
Loss: 12.873652281135488
Info at epoch #5000:
Loss: 12.871152623006394


![Linear regression 3D model with 100 hidden neuron and 5000 epochs](./plots/linear_regression_3d_100_5000_ridge+lsc.png)